In [74]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np

In [75]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

model = loadGloveModel('/home/prashant/Downloads/kagsa/data/glove.840B.300d.txt')
#vocab = model.keys()

Loading Glove Model
Done. 2196016  words loaded!


In [77]:
def get_vector(model,all_data):
    vector_sen = []
    for d in all_data:
        single_sen_vec = []
        words = d.split(' ')
        for w in words:
            try:
                
                get_word_vec = model[w]
            except:
                get_word_vec =np.zeros(300)
                pass
            single_sen_vec.append(get_word_vec)
        v = np.array(single_sen_vec).mean(axis=0)
        vector_sen.append(v)
    return vector_sen

In [2]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [3]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [4]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_train['tag'] = 'train'
print df_train.shape

(43266, 10)


In [7]:
df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
df_test['tag'] = 'test'
print df_test.shape

(18543, 9)


In [202]:
df = pd.concat([df_train , df_test],axis= 0)
print df.shape
df.head()

(61809, 10)


/home/prashant/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Company-response,Complaint-ID,Complaint-Status,Complaint-reason,Consumer-complaint-summary,Consumer-disputes,Date-received,Date-sent-to-company,Transaction-Type,tag
0,NaN,Tr-1,Closed with explanation,"Loan servicing, payments, escrow account","Seterus, Inc a déposé un faux rapport auprès d...",Yes,11/11/2015,11/11/2015,Mortgage,train
1,Company chooses not to provide a public response,Tr-2,Closed with non-monetary relief,Incorrect information on credit report,XX / XX / XXXX La requête en faillite n ° XXXX...,No,7/7/2015,7/7/2015,Credit reporting,train
2,NaN,Tr-3,Closed with explanation,Using a debit or ATM card,"El XXXX / XXXX / 15, estaba preparando el vuel...",No,5/7/2015,5/7/2015,Bank account or service,train
3,Company believes it acted appropriately as aut...,Tr-4,Closed with explanation,Cont'd attempts collect debt not owed,"The loan was paid in XXXX XXXX. In XXXX, 4 yea...",No,11/12/2016,11/12/2016,Debt collection,train
4,Company has responded to the consumer and the ...,Tr-5,Closed with explanation,Payoff process,J'ai obtenu un compte de crédit de soins pour ...,No,9/29/2016,9/29/2016,Credit card,train


In [205]:
df.loc[df['Consumer-disputes']=='Yes']

,Company-response,Complaint-ID,Complaint-Status,Complaint-reason,Consumer-complaint-summary,Consumer-disputes,Date-received,Date-sent-to-company,Transaction-Type,tag
0,NaN,Tr-1,Closed with explanation,"Loan servicing, payments, escrow account","Seterus, Inc a déposé un faux rapport auprès d...",Yes,11/11/2015,11/11/2015,Mortgage,train
5,NaN,Tr-6,Closed with explanation,"Loan modification,collection,foreclosure",The owner of my original mortgage filed for ba...,Yes,8/2/2016,8/3/2016,Mortgage,train
8,NaN,Tr-9,Closed with explanation,Cont'd attempts collect debt not owed,Una agencia de cobranza me hizo adulterar de q...,Yes,1/18/2016,1/18/2016,Debt collection,train
24,NaN,Tr-25,Closed with explanation,Incorrect information on credit report,In XX / XX / XXXX I pulled out my Equifax cred...,Yes,3/10/2017,3/10/2017,Credit reporting,train
26,Company believes it acted appropriately as aut...,Tr-27,Closed with explanation,Deposits and withdrawals,Abrí una cuenta de ahorros y cheques para jóve...,Yes,1/31/2016,1/31/2016,Bank account or service,train
31,Company chooses not to provide a public response,Tr-32,Closed with explanation,Settlement process and costs,We request a simplified refinancing XXXX XXXX ...,Yes,5/26/2015,5/29/2015,Mortgage,train
47,Company chooses not to provide a public response,Tr-48,Closed with explanation,"Loan servicing, payments, escrow account",Le prêt immobilier a été refinancé le XXXX / X...,Yes,5/1/2015,5/1/2015,Mortgage,train
52,Company chooses not to provide a public response,Tr-53,Closed with explanation,"Loan servicing, payments, escrow account",The FHA XXXX refinancing was completed with ap...,Yes,11/15/2015,11/18/2015,Mortgage,train
57,NaN,Tr-58,Closed with explanation,False statements or representation,Portfolio Recovery Associates me demandó en la...,Yes,9/10/2015,9/11/2015,Debt collection,train
59,NaN,Tr-60,Closed with explanation,Using a debit or ATM card,XXXX has installed chip readers in their boxes...,Yes,5/3/2016,5/6/2016,Bank account or service,train


In [144]:
#df_train.isnull().sum()
df['Transaction-Type'] = df['Transaction-Type'].apply(preprocess_data)

In [145]:
# vectors =  get_vector(model,list(df['Transaction-Type']))
text = list(df['Transaction-Type'])
vectorizer = TfidfVectorizer(min_df=1)
# # tokenize and build vocab
vectorizer.fit(text)
#vector = get_vector(model,list(df['merge']))

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [147]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import silhouette_samples, silhouette_score
# from sklearn.cluster import KMeans

# kmeans = KMeans(n_clusters=100 , max_iter=300 , n_init=100 , random_state=0).fit(vectors)

# pred = kmeans.predict(vectors)

# labels = list(map(lambda x : str(x),kmeans.labels_))
# # encode document
vectors = vectorizer.transform(text)
# # summarize encoded vector
print(vectors.shape)


(61809, 27)


In [59]:
# tran_type = {'Debt collection' : 'd' ,
#              'Credit reporting':'c' ,
#              'Credit card':'c',
#              'Credit reporting, credit repair services, or other personal consumer reports':'c',
#              'Credit card or prepaid card':'c',\
#              'Prepaid card':'c',
#              'Mortgage':'l',
#              'Student loan':'l',
#              'Consumer Loan':'l',
#              'Payday loan':'l',
#              'Vehicle loan or lease':'l',\
#              'Payday loan, title loan, or personal loan':'l',\
#              'Virtual currency':'c',\
#              'Money transfer, virtual currency, or money service':'c',\
#              'Money transfers':'b',\
#              'Bank account or service':'b',\
#              'Checking or savings account':'b',
#              'Other financial service':'o'
#     }
# len(tran_type)
# replace = []
# for t in df['Transaction-Type']:
#     replace.append(tran_type[t])
    
# df['Transaction-Type'] = replace

In [148]:
vec_df = pd.DataFrame(vectors.toarray())


In [206]:
vec_df['Complaint-ID'] = list(df['Complaint-ID'])
vec_df['tag'] = list(df['tag'])
vec_df['Complaint-Status'] = list(df['Complaint-Status'])
vec_df['Consumer-disputes'] = list(df['Consumer-disputes'])
df = vec_df
#df = df[['Complaint-ID','tag','Complaint-Status']]
#df['labels'] =  labels

In [126]:
vec_df.head()

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,Complaint-ID,tag,Complaint-Status
0,-0.936010,0.502420,-0.768120,0.039594,0.265520,-0.546890,0.000788,-0.504590,0.567610,2.752700,...,0.396500,0.133500,-0.52315,0.308890,-0.310620,-0.085346,-0.495020,Tr-1,train,Closed with explanation
1,-0.800900,-0.017153,-0.304135,0.200220,-0.204915,-0.006370,-0.157285,-0.271715,0.064239,2.621350,...,0.350540,0.271489,-0.76521,-0.208305,0.062628,0.024445,0.101879,Tr-2,train,Closed with non-monetary relief
2,-0.219517,0.390540,-0.068047,-0.088947,-0.006600,-0.083402,-0.016224,-0.248142,0.036399,2.613733,...,-0.031887,0.258049,-0.52715,0.150105,-0.050632,-0.273937,-0.039494,Tr-3,train,Closed with explanation
3,-0.691325,-0.088485,-0.114577,-0.227409,-0.076076,-0.313092,0.072820,-0.244675,0.257785,2.255850,...,0.307510,0.368490,-0.65840,0.558440,-0.404740,-0.395665,-0.446500,Tr-4,train,Closed with explanation
4,-0.823445,-0.006205,-0.207670,-0.371010,0.418105,-0.054205,-0.366800,-0.316295,-0.020920,2.100250,...,-0.033655,0.555210,-0.14298,0.516980,0.048301,0.007050,0.370015,Tr-5,train,Closed with explanation


In [208]:
df_cat = df[['Consumer-disputes']]
df_one = pd.get_dummies(df_cat)

In [209]:
df_one['Complaint-ID'] = df['Complaint-ID']
df_one['tag'] = df['tag']

In [210]:
df_one['Complaint-Status'] = df['Complaint-Status']

In [211]:
complete_df_one_hot = df_one

train_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'train']
train_data.drop(['tag'],inplace=True,axis=1)
test_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'test']
test_data.drop(['tag'],inplace=True,axis =1)

In [212]:
y_train = train_data['Complaint-Status']
x_train = train_data.drop(['Complaint-Status','Complaint-ID'],axis =1)

In [213]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train)

In [214]:
nb = LogisticRegression().fit(X_train, y_train_internal)
pred_internal = nb.predict(X_test)
import sklearn
sklearn.metrics.f1_score(pred_internal,y_test_internal,average='weighted')
# '''
# # Import the model we are using
# from sklearn.ensemble import RandomForestClassifier

# # Instantiate model with 1000 decision trees
# rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

# # Train the model on training data
# rm = rf.fit(X_train, y_train_internal)
# pred_internal = rm.predict(X_test)
# sklearn.metrics.f1_score(pred_internal,y_test_internal,average='weighted')
# '''

0.88282984766331

In [215]:
id_ = list(test_data['Complaint-ID'])


In [216]:
print train_data.shape
test_data.shape
test_data.drop(['Complaint-Status','Complaint-ID'],inplace=True,axis=1)

(43266, 4)


In [161]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
43266,0.600053,0.624193,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.500319,0.000000,0.0,0.0,0.0,0.0
43267,0.000000,0.000000,0.0,0.0,0.707107,0.0,0.000000,0.0,0.707107,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
43268,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.643263,0.0,0.000000,0.0,...,0.0,0.0,0.765646,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
43269,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.773268,0.0,0.0,0.0,0.0
43270,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.643263,0.0,0.000000,0.0,...,0.0,0.0,0.765646,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [217]:
test_data.head()
y_pred = nb.predict(test_data)

In [218]:
result = pd.DataFrame(id_,columns=['Complaint-ID'])
result['Complaint-Status'] = y_pred

In [219]:
result.to_csv('/home/prashant/Downloads/c3cc8568-0-dataset/one_tfidf_CD_embed.csv',index=None)